In [2]:
import numpy as np
import torch
from torch import nn
import torch.optim as optim
import matplotlib.pyplot as plt
from lstm_autoencoder_source import CNN_Autoencoder
from resiudalBlock import ResidualBlock
from resiudalBlock import ResNetAutoencoder
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os
import cv2
import pickle

In [ ]:
kind = 'squat'
with open(f"{kind}_dataloader.pkl", "rb") as f:
    saved_data = pickle.load(f)

loaded_dataset = saved_data["dataset"]
dataloader = DataLoader(
    loaded_dataset,
    batch_size=saved_data["batch_size"],
    shuffle=saved_data["shuffle"],
)

In [ ]:
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('mps') if torch.mps.is_available() else torch.device('cpu')
model = ResNetAutoencoder().to(device)
def saveModel():
    torch.save(model.state_dict(), f'model/{kind}_train_l2.pt')
criterion = nn.MSELoss()
optimizer = torch.optim.RAdam(model.parameters(), lr=1e-2)
scheduler = CosineAnnealingLR(optimizer,T_max= 50, eta_min=1e-4)
# 학습 루프
epochs = 200
min_loss = 1e+10
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for batch in dataloader:
    # batch의 형태에 맞게 x와 y 추출
        x, y = batch
    # x와 y를 이용한 처리
        input = x.to(device) 
          
        optimizer.zero_grad()

        # 순전파
        output = model(input)

        # 손실 계산
        loss = criterion(output,y.to(device))
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    mean_loss =running_loss / len(dataloader)
    print(f"Epoch [{epoch + 1}/{epochs}], Loss: {running_loss / len(dataloader)}")
    if min(mean_loss,min_loss) == mean_loss:
        min_loss = mean_loss
        print('model_saved')
        saveModel()
    

Epoch [1/200], Loss: 0.018127277968451382
model_saved
Epoch [2/200], Loss: 0.0031055378541350364
model_saved
Epoch [3/200], Loss: 0.003095915885642171
model_saved
Epoch [4/200], Loss: 0.003080816031433642
model_saved
Epoch [5/200], Loss: 0.0031653994601219893
Epoch [6/200], Loss: 0.0031307853711768983
Epoch [7/200], Loss: 0.0031608038768172263
Epoch [8/200], Loss: 0.003075997801497579
model_saved
Epoch [9/200], Loss: 0.0030589358787983657
model_saved
Epoch [10/200], Loss: 0.0030600260011851787
Epoch [11/200], Loss: 0.003123523141257465
Epoch [12/200], Loss: 0.0030352429347112776
model_saved
Epoch [13/200], Loss: 0.003018816439434886
model_saved
Epoch [14/200], Loss: 0.0030226332461461424
Epoch [15/200], Loss: 0.0030265721213072537
Epoch [16/200], Loss: 0.003007401670329273
model_saved
Epoch [17/200], Loss: 0.0029896590718999504
model_saved
Epoch [18/200], Loss: 0.0029658167343586685
model_saved
Epoch [19/200], Loss: 0.0029576249700039624
model_saved
Epoch [20/200], Loss: 0.002960604042